Adapted from https://stackoverflow.com/questions/14622526/create-a-table-from-query-results-in-google-bigquery

I could only get this to work in Python 3 for whatever reason.

# Create client

In [ ]:
from google.cloud import bigquery

class Client(object):

    def __init__(self, query, destination_dataset, destination_table):
        """
        A Client that performs a user-specified query and INSERTS the results in a
        user-specified location.

        All init args are strings. Note that the destination project is the
        default project from your Google Cloud configuration.
        """
        self.query = query
        #self.project = origin_project
        #self.dataset = origin_dataset
        #self.table = origin_table
        self.dest_dataset = destination_dataset
        self.dest_table_name = destination_table
        self.client = bigquery.Client()

    def run(self):
        query = self.query #query = ("SELECT * FROM `{project}.{dataset}.{table}`;".format(
        #    project=self.project, dataset=self.dataset, table=self.table))

        job_config = bigquery.QueryJobConfig()

        # Set configuration.query.destinationTable
        destination_dataset = self.client.dataset(self.dest_dataset)
        destination_table = destination_dataset.table(self.dest_table_name)
        job_config.destination = destination_table

        # Set configuration.query.createDisposition
        job_config.create_disposition = 'CREATE_IF_NEEDED'

        # Set configuration.query.writeDisposition
        job_config.write_disposition = 'WRITE_TRUNCATE'

        # Start the query
        job = self.client.query(query, job_config=job_config)

        # Wait for the query to finish
        job.result()

## Make antibiotic prescribing table

In [ ]:
q = '''
SELECT
  practice,
  pct,
  bnf_code,
  bnf_name,
  month,
  items,
  quantity,
  actual_cost
FROM
  ebmdatalab.hscic.normalised_prescribing_standard
WHERE
  (bnf_code LIKE '0501%%')
'''
query = Client(query=q, destination_dataset='alex', destination_table='antibiotic_prescribing')
query.run()

## Match BNF chemical level with ADQ table

In [ ]:
q = '''
WITH
  subQbnf AS (
  SELECT
    *,
    REGEXP_EXTRACT(chemical, r'^([\w\-]+)') AS bnf_chemical
  FROM
    ebmdatalab.hscic.bnf),
  subQadq AS (
  SELECT
    *,
    REGEXP_EXTRACT(BNF_Name, r'^([\w\-]+)') AS adq_chemical
  FROM
    ebmdatalab.alex.antibiotic_adq)
SELECT
  DISTINCT SUBSTR(p.bnf_code,1,9) AS chemical_code,
  Admin_Route,
  ADQ,
  Unit
FROM
  ebmdatalab.alex.antibiotic_prescribing p
LEFT JOIN
  subQbnf
ON
  SUBSTR(bnf_code,1,9) = subQbnf.chemical_code
LEFT JOIN
  subQadq
ON
  bnf_chemical = adq_chemical
'''
query = Client(query=q, destination_dataset='alex', destination_table='antibiotic_adq_codes')
query.run()

## Regex dose info

In [ ]:
q = '''
SELECT
  chemical,
  presentation_code,
  presentation,
  REGEXP_EXTRACT(presentation, r'[^\d]+(\d*\,?\.?\d*)') AS dose,
  REGEXP_EXTRACT(presentation, r'[^\d]+\d*\,?\.?\d*([\w]*)') AS dose_units,
  REGEXP_EXTRACT(presentation, r'\d+\,?\.?\d*.*?/(\d*\.?\d*)') AS per,
  REGEXP_EXTRACT(presentation, r'\d+\,?\.?\d*.*?/\d+\.?\d*([\w]*)') AS per_units,
  CASE
    WHEN presentation LIKE '%Pfs%' OR presentation LIKE '%Amp%' OR presentation LIKE '%Imp%' THEN TRUE
    ELSE FALSE
  END AS fixed_dose,
  REGEXP_EXTRACT(presentation, r'[^_]+_(.*?)(?:[0-9]+.*)?$') AS delivery_method
FROM
  ebmdatalab.hscic.bnf
WHERE
  presentation_code LIKE '0501%'
ORDER BY
  presentation_code
'''
query = Client(query=q, destination_dataset='alex', destination_table='cost_%s')
query.run()

## Smoosh together